In [13]:
import pandas as pd
import numpy as np

loc = 'C:/Users/sagic/OneDrive/المستندات/Projects/UFC_Card_Predictor/'
file1 = 'raw_fighter_details.csv'
file2 = 'raw_total_fight_data.csv'
df1 = pd.read_csv(loc+file1)
df2 = pd.read_csv(loc+file2,delimiter = ';')

# Feature Engineering
****

## df1(=fighter's data) feature engineering

In [14]:
df = df1

df = df.rename(columns = {'fighter_name':'name','Height':'height','Weight':'weight','Reach':'reach','Stance':'stance','DOB':'age'})
print(df.shape)
df.head(5)

(3313, 6)


,name,height,weight,reach,stance,age
0,AJ Fonseca,"5' 4""",145 lbs.,NaN,NaN,NaN
1,AJ Matthews,"5' 11""",185 lbs.,NaN,NaN,NaN
2,AJ McKee,"5' 10""",145 lbs.,NaN,NaN,NaN
3,AJ Siscoe,"5' 7""",135 lbs.,NaN,NaN,NaN
4,Aalon Cruz,"6' 0""",145 lbs.,NaN,NaN,NaN


In [15]:
df['height'] = df['height'].apply(lambda x: str(x) )
df['height'] = df['height'].apply(lambda x: '0' if x == 'nan' else str(x))
df['height'] = df['height'].apply(lambda x: 30.48*float(x[0]) if len(x) == 1 else 30.48*float(x[0])+ 2.54*float(x[x.find("'")+1:x.find('"')]) ) 

# height
height_list = df['height'][df['height']!=0.0]
mean_height = np.round(np.mean(height_list),2)
df['height'] = df['height'].apply(lambda x: mean_height if x==0.00 else x )

# weight
df['weight'] = df['weight'].apply(lambda x: 0.45*float(str(x)[0:3]) )
mean_weight = np.round( np.mean(df['weight']) )
df['weight'] = df['weight'].apply(lambda x: mean_weight if str(x)=='nan' else x )

# reach
df['reach'] = df['reach'].apply( lambda x: str(x))
df['reach'] = df['reach'].apply( lambda x: '00"' if x=='nan' else x)
df['reach'] = df['reach'].apply( lambda x: float(x[:x.find('"')]) )

df['reach/height'] = np.round(df['reach']/df['height'],2)
reach_height_ratio = np.mean(df['reach/height'])
#df['reach'] = df['reach'].apply(lambda x: x['height'] if x==0.0 else x ) # replacing null reach with: height*mean(reach/height) 

# age
from datetime import date,timedelta
current_date = date.today().year
df.age = df.age.apply(lambda x: str(x))
df.age = df.age.apply(lambda x: x[len(x)-4:len(x)])
df.age = df.age.apply(lambda x: 0 if x=='n' else int(x))
year_mean = np.round(np.mean(df.age[df['age']!=0.0]),0)
df.age = df.age.apply(lambda x: current_date-x if x!=0.0 else current_date-year_mean )

df.head(5)

,name,height,weight,reach,stance,age,reach/height
0,AJ Fonseca,162.56,65.25,0.0,NaN,39.0,0.0
1,AJ Matthews,180.34,83.25,0.0,NaN,39.0,0.0
2,AJ McKee,177.80,65.25,0.0,NaN,39.0,0.0
3,AJ Siscoe,170.18,60.75,0.0,NaN,39.0,0.0
4,Aalon Cruz,182.88,65.25,0.0,NaN,39.0,0.0


### find max stance & fixing null stance

In [16]:
print( 'Orthodox = %s' %df['stance'][df['stance']=='Orthodox'].count())
print( 'Southpaw = %s' %df['stance'][df['stance']=='Southpaw'].count())
print( 'Switch = %s' %df['stance'][df['stance']=='Switch'].count())
print( 'Open Stance = %s' %df['stance'][df['stance']=='Open Stance'].count())
print( 'Sideways = %s' %df['stance'][df['stance']=='Sideways'].count())

Orthodox = 1913
Southpaw = 431
Switch = 85
Open Stance = 7
Sideways = 3


In [17]:
df['stance'] = df['stance'].apply( lambda x: 'Orthodox' if str(x)=='nan' else x )
df.head(5)

,name,height,weight,reach,stance,age,reach/height
0,AJ Fonseca,162.56,65.25,0.0,Orthodox,39.0,0.0
1,AJ Matthews,180.34,83.25,0.0,Orthodox,39.0,0.0
2,AJ McKee,177.80,65.25,0.0,Orthodox,39.0,0.0
3,AJ Siscoe,170.18,60.75,0.0,Orthodox,39.0,0.0
4,Aalon Cruz,182.88,65.25,0.0,Orthodox,39.0,0.0


## df2(=past fights) Feature Engineering

In [18]:
df = df2
df = df.rename(columns = {'Fight_type':'category','Winner':'win'})

### striking features eng

In [19]:
df['r_total_throws'] = df['R_TOTAL_STR.'].apply(lambda x: int(x[(len(x)-x.find('f')+1):len(x)] ))
df['b_total_throws'] = df['B_TOTAL_STR.'].apply(lambda x: int(x[(len(x)-x.find('f')+1):len(x)] ))

df['r_total_strikes'] = df['R_TOTAL_STR.'].apply(lambda x: int(x[:x.find(' ')]))
df['b_total_strikes'] = df['B_TOTAL_STR.'].apply(lambda x: int(x[:x.find(' ')]))

df['r_sig_strikes'] = df['R_SIG_STR.'].apply(lambda x: int(x[:x.find(' ')]))
df['b_sig_strikes'] = df['B_SIG_STR.'].apply(lambda x: int(x[:x.find(' ')]))

df['r_sig_str_pct'] = df['R_SIG_STR_pct'].apply(lambda x: int(x[:x.find('%')])/100.0)
df['b_sig_str_pct'] = df['B_SIG_STR_pct'].apply(lambda x: int(x[:x.find('%')])/100.0)

df = df.rename(columns = {'R_KD':'r_knockdowns','B_KD':'b_knockdowns'})

### takedowns features eng

In [20]:
df['r_total_takedowns'] = df['R_TD'].apply(lambda x: int(x[x.find('f')+1:len(x)]))
df['b_total_takedowns'] = df['B_TD'].apply(lambda x: int(x[x.find('f')+1:len(x)]))

df['r_succ_takedowns'] = df['R_TD'].apply(lambda x: int(x[:x.find(' ')]))
df['b_succ_takedowns'] = df['B_TD'].apply(lambda x: int(x[:x.find(' ')]))

df['r_succ_takedowns_pct'] = df['R_TD_pct'].apply(lambda x: int(x[:x.find('%')])/100.0)
df['b_succ_takedowns_pct'] = df['B_TD_pct'].apply(lambda x: int(x[:x.find('%')])/100.0)

### grappling features eng

In [21]:
df = df.rename(columns = {'R_SUB_ATT':'r_sub_attemp','B_SUB_ATT':'b_sub_attemp'})
df = df.rename(columns = {'R_PASS':'r_pass','B_PASS':'b_pass'})
df = df.rename(columns = {'R_REV':'r_rev','B_REV':'b_rev'})

In [22]:
df.to_csv('UFC_feature_file.csv')

In [23]:
def getWinnerColor(i):

    '''return winner by color'''
    
    r_fighter = df.loc[i,'R_fighter']
    b_fighter = df.loc[i,'B_fighter']
    winner = df.loc[i,'win']
    
    #print('row = %i, red = %s, blue = %s, win = %s' %(i,b_fighter,r_fighter,winner))
    
    if winner == r_fighter:
        return 'r_fighter'
        
    elif winner == b_fighter:
        return 'b_fighter'
    
    else:
        return 'none'
       

## Main Features updating loop

In [24]:
for i in range(len(df)):
    df.loc[i,'win'] = getWinnerColor(i)

print("df['win'] updated")    

df['win'] updated


In [25]:
df.win[:15]

0     r_fighter
1     r_fighter
2     r_fighter
3     b_fighter
4     b_fighter
5     r_fighter
6     r_fighter
7     b_fighter
8     b_fighter
9     r_fighter
10    b_fighter
11    r_fighter
12    r_fighter
13    b_fighter
14    b_fighter
Name: win, dtype: object

In [26]:
r_fighter = df.loc[2,'R_fighter']
b_fighter = df.loc[2,'B_fighter']
winner  = df.loc[2,'win']

print('row = %i, red = %s, blue = %s, win = %s' %(i,b_fighter,r_fighter,winner))

r_fighter==winner

row = 5143, red = Donald Cerrone, blue = Tony Ferguson, win = r_fighter


False

In [27]:
df = df['R_fighter', 'B_fighter', 'r_knockdowns', 'b_knockdowns','r_sub_attemp',
       'b_sub_attemp', 'r_pass', 'b_pass', 'r_rev', 'b_rev', 'win_by',
       'last_round', 'last_round_time', 'Format', 'Referee', 'date',
       'location', 'category', 'win', 'r_total_throws', 'b_total_throws',
       'r_total_strikes', 'b_total_strikes', 'r_sig_strikes', 'b_sig_strikes',
       'r_sig_str_pct', 'b_sig_str_pct', 'r_total_takedowns',
       'b_total_takedowns', 'r_succ_takedowns', 'b_succ_takedowns',
       'r_succ_takedowns_pct', 'b_succ_takedowns_pct']

KeyError: ('R_fighter', 'B_fighter', 'r_knockdowns', 'b_knockdowns', 'r_sub_attemp', 'b_sub_attemp', 'r_pass', 'b_pass', 'r_rev', 'b_rev', 'win_by', 'last_round', 'last_round_time', 'Format', 'Referee', 'date', 'location', 'category', 'win', 'r_total_throws', 'b_total_throws', 'r_total_strikes', 'b_total_strikes', 'r_sig_strikes', 'b_sig_strikes', 'r_sig_str_pct', 'b_sig_str_pct', 'r_total_takedowns', 'b_total_takedowns', 'r_succ_takedowns', 'b_succ_takedowns', 'r_succ_takedowns_pct', 'b_succ_takedowns_pct')

In [236]:
df.columns

Index(['R_fighter', 'B_fighter', 'r_knockdowns', 'b_knockdowns', 'R_SIG_STR.',
       'B_SIG_STR.', 'R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TOTAL_STR.',
       'B_TOTAL_STR.', 'R_TD', 'B_TD', 'R_TD_pct', 'B_TD_pct', 'r_sub_attemp',
       'b_sub_attemp', 'r_pass', 'b_pass', 'r_rev', 'b_rev', 'R_HEAD',
       'B_HEAD', 'R_BODY', 'B_BODY', 'R_LEG', 'B_LEG', 'R_DISTANCE',
       'B_DISTANCE', 'R_CLINCH', 'B_CLINCH', 'R_GROUND', 'B_GROUND', 'win_by',
       'last_round', 'last_round_time', 'Format', 'Referee', 'date',
       'location', 'category', 'win', 'r_total_throws', 'b_total_throws',
       'r_total_strikes', 'b_total_strikes', 'r_sig_strikes', 'b_sig_strikes',
       'r_sig_str_pct', 'b_sig_str_pct', 'r_total_takedowns',
       'b_total_takedowns', 'r_succ_takedowns', 'b_succ_takedowns',
       'r_succ_takedowns_pct', 'b_succ_takedowns_pct'],
      dtype='object')

In [108]:
def getWinnerColor(df):
    '''return winner color instead of name'''
    
    pass

,f1name,f2name,event_org,event_date,event_place,f1result,f2result,method,method_d,ref,round,time,birth_date,height,weight,association,class,country,locality,winner
0,Royce Gracie,Gerard Gordeau,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,Submission,Rear-Naked Choke,Helio Vigio,1,1:44,12/12/1966,72.0,176.0,Gracie Humaita,Middleweight,Brazil,Rio de Janeiro,Royce Gracie
1,Royce Gracie,Gerard Gordeau,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,Submission,Rear-Naked Choke,Helio Vigio,1,1:44,12/12/1966,72.0,176.0,Gracie Humaita,Middleweight,Brazil,Rio de Janeiro,Royce Gracie
2,Royce Gracie,Ken Shamrock,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,Submission,Rear-Naked Choke,Helio Vigio,1,0:57,12/12/1966,72.0,176.0,Gracie Humaita,Middleweight,Brazil,Rio de Janeiro,Royce Gracie
3,Royce Gracie,Ken Shamrock,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,Submission,Rear-Naked Choke,Helio Vigio,1,0:57,12/12/1966,72.0,176.0,Gracie Humaita,Middleweight,Brazil,Rio de Janeiro,Royce Gracie
4,Royce Gracie,Art Jimmerson,Ultimate Fighting Championship,11/12/1993,"McNichols Arena, Denver, Colorado, United States",win,loss,Submission,Smother Choke,Joao Alberto Barreto,1,2:18,12/12/1966,72.0,176.0,Gracie Humaita,Middleweight,Brazil,Rio de Janeiro,Royce Gracie


In [109]:
df = df[['f1name','f2name','winner','event_date','event_place','method','method_d','ref','round','time','birth_date','height','weight','association','class','country','locality']]
df = df.sort_values(by=['event_date'], ascending=True)
print(df.shape)
df.head()

(3678, 17)


,f1name,f2name,winner,event_date,event_place,method,method_d,ref,round,time,birth_date,height,weight,association,class,country,locality
2052,Brad Tavares,Phil Baroni,Brad Tavares,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",KO,Knees and Punches,Josh Rosenthal,1,4:20,12/21/1987,71.0,185.0,Xtreme Couture,Middleweight,United States,"Kailua, Hawaii"
1190,Frankie Edgar,Gray Maynard,Frankie Edgar,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",Draw,Split,Yves Lavigne,5,5:00,10/16/1981,66.0,145.0,Ricardo Almeida Jiu-Jitsu,Featherweight,United States,"Toms River, New Jersey"
1390,Jeremy Stephens,Marcus Davis,Jeremy Stephens,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",KO,Punch,Kim Winslow,3,2:33,5/26/1986,69.0,145.0,Alliance MMA,Featherweight,United States,"Des Moines, Iowa"
1105,Clay Guida,Takanori Gomi,Clay Guida,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",Submission,Guillotine Choke,Josh Rosenthal,2,4:27,12/8/1981,67.0,145.0,Jackson-Wink MMA,Featherweight,United States,"Round Lake, Illinois"
1509,Dong Hyun Kim,Nate Diaz,Dong Hyun Kim,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",Decision,Unanimous,Yves Lavigne,3,5:00,11/17/1981,73.0,170.0,Busan Team M.A.D.,Welterweight,South Korea,"Busan, Yeongnam"


# Feature Engineering

In [114]:
def getWinsPer(name):
    
    temp = df[(df['f1name']==name) | (df['f2name']==name)]
    fights = len(temp)
    
    temp = temp[temp['winner']==name]
    wins = len(temp)
    
    if fights != 0:
        return wins/fights
    else:
        return 0 

In [115]:
def getAssocPer(assoc):
    
    temp = df[df['association']==assoc]
    fights = len(temp)

    temp = temp[temp['winner']==temp['f1name']]
    wins = len(temp)

    if fights != 0:
        return wins/fights
    else:
        return 0

In [116]:
df = df.reset_index(drop=True)
df.tail()

,f1name,f2name,winner,event_date,event_place,method,method_d,ref,round,time,birth_date,height,weight,association,class,country,locality,win_w,association_w
3673,Royce Gracie,Kimo Leopoldo,Royce Gracie,9/9/1994,"Grady Cole Center, Charlotte, North Carolina, ...",Submission,Armlock,John McCarthy,1,4:40,12/12/1966,72.0,176.0,Gracie Humaita,Middleweight,Brazil,Rio de Janeiro,,
3674,Steve Jennum,Harold Howard,Steve Jennum,9/9/1994,"Grady Cole Center, Charlotte, North Carolina, ...",Submission,Punches,John McCarthy,1,1:27,NaN,70.0,215.0,NaN,Heavyweight,United States,"Omaha, Nebraska",,
3675,Harold Howard,Roland Payne,Harold Howard,9/9/1994,"Grady Cole Center, Charlotte, North Carolina, ...",KO,Punch,John McCarthy,1,0:46,NaN,74.0,240.0,NaN,Heavyweight,Canada,"Niagara Falls, Ontario",,
3676,Keith Hackney,Emmanuel Yarborough,Keith Hackney,9/9/1994,"Grady Cole Center, Charlotte, North Carolina, ...",TKO,Punches,John McCarthy,1,1:59,4/15/1958,71.0,200.0,NaN,Light Heavyweight,United States,"Addison, Illinois",,
3677,Royce Gracie,Kimo Leopoldo,Royce Gracie,9/9/1994,"Grady Cole Center, Charlotte, North Carolina, ...",Submission,Armlock,John McCarthy,1,4:40,12/12/1966,72.0,176.0,Gracie Humaita,Middleweight,Brazil,Rio de Janeiro,,


In [117]:
# main loop


df['win_w']=''
df['association_w']=''

for i in range(len(df)):
    
    name = df.loc[i,'f1name']
    assoc = df.loc[i,'association']
    
    df.loc[i,'win_w'] = getWinsPer(name)
    df.loc[i,'association_w'] = getAssocPer(assoc)

In [119]:
df.head(20)

,f1name,f2name,winner,event_date,event_place,method,method_d,ref,round,time,birth_date,height,weight,association,class,country,locality,win_w,association_w
0,Brad Tavares,Phil Baroni,Brad Tavares,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",KO,Knees and Punches,Josh Rosenthal,1,4:20,12/21/1987,71.0,185.0,Xtreme Couture,Middleweight,United States,"Kailua, Hawaii",0.666667,1.0
1,Frankie Edgar,Gray Maynard,Frankie Edgar,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",Draw,Split,Yves Lavigne,5,5:00,10/16/1981,66.0,145.0,Ricardo Almeida Jiu-Jitsu,Featherweight,United States,"Toms River, New Jersey",0.789474,1.0
2,Jeremy Stephens,Marcus Davis,Jeremy Stephens,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",KO,Punch,Kim Winslow,3,2:33,5/26/1986,69.0,145.0,Alliance MMA,Featherweight,United States,"Des Moines, Iowa",0.5,1.0
3,Clay Guida,Takanori Gomi,Clay Guida,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",Submission,Guillotine Choke,Josh Rosenthal,2,4:27,12/8/1981,67.0,145.0,Jackson-Wink MMA,Featherweight,United States,"Round Lake, Illinois",0.545455,1.0
4,Dong Hyun Kim,Nate Diaz,Dong Hyun Kim,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",Decision,Unanimous,Yves Lavigne,3,5:00,11/17/1981,73.0,170.0,Busan Team M.A.D.,Welterweight,South Korea,"Busan, Yeongnam",0.75,1.0
5,Brian Stann,Chris Leben,Brian Stann,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",TKO,Knee and Punches,Josh Rosenthal,1,3:37,9/24/1980,73.0,185.0,Jackson-Wink MMA,Middleweight,United States,"Jacksonville, North Carolina",0.545455,1.0
6,Jacob Volkmann,Antonio McKee,Jacob Volkmann,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",Decision,Split,Steve Mazzagatti,3,5:00,9/5/1980,69.0,170.0,Minnesota Martial Arts Academy,Welterweight,United States,"White Bear Lake, Minnesota",0.6,1.0
7,Daniel Roberts,Greg Soto,Daniel Roberts,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",Submission,Kimura,Kim Winslow,1,3:45,8/5/1980,70.0,170.0,Cesar Gracie Fight Team,Welterweight,United States,"Rockford, Illinois",0.428571,1.0
8,Thiago Silva,Brandon Vera,Thiago Silva,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",NC,Overturned by NSAC,Steve Mazzagatti,3,5:00,11/12/1982,73.0,208.0,Blackzilians,Heavyweight,Brazil,Sao Paulo,0.75,1.0
9,Diego Nunes,Mike Thomas Brown,Diego Nunes,1/1/2011,"MGM Grand Garden Arena, Las Vegas, Nevada, Uni...",Decision,Split,Yves Lavigne,3,5:00,11/30/1982,66.0,145.0,X-Gym,Featherweight,Brazil,"Caxias do Sul, Rio Grande do Sul",0.5,1.0


In [17]:
name = 'Nate Diaz'
def getMethodPer()
    temp = df2[(df2['f1name']==name)|(df2['f2name']==name)]
    fights = len(temp)
    wins = len(df2[df2['winner']==name])
    df2['win_w'] = wins/fights

In [104]:
def getDictByName(dict):
    dictByNames = {}
    for name in dict:
        print(name)
                
    

dict = {
  "name1": "file1.txt",
  "name2": "file2.txt",
  "name1": "file3.txt",
  "name1": "file4.txt",
  "name2": "file5.txt",
  "name2": "file6.txt",
  "name1": "file66.txt",
}


getDictByName(dict)

name1
name2


In [92]:
string = 'Tutorials Point is best'
for i in ''.join(string.split()):
   print(i, end= ''. '')

SyntaxError: invalid syntax (<ipython-input-92-f2f630647caa>, line 3)

In [69]:
list('DSFGDSFG')

['D', 'S', 'F', 'G', 'D', 'S', 'F', 'G']